In [1]:
# Load Packages
import os
import pandas as pd
import numpy as np
import random

In [2]:
#Load the data from github
url = 'https://raw.githubusercontent.com/EktaDhobley/Algorithms-in-the-wild/main/data/unmodified_as_is_data.csv'
df = pd.read_csv(url, index_col = False)
display(df)

,hall_id,building_id,floor_no,room_no,occupancy,room_type,floor_plan,price,type,name,...,has_microfridge,has_curtains,has_tv_in_room,has_wifi_in_room,has_carpeted_flooring,has_wardrobe,has_twin_bed_mattress,has_hardwood_flooring,has_accessibility_ramps,has_private_bathroom
0,AH,NaN,Floor 1,101,1,1 bedroom,Traditional Single,"8,416",Traditional Residence Hall,Allen Hall,...,1,1,1,1,1,1,1,0,0,0
1,AH,NaN,Floor 1,102,1,1 bedroom,Traditional Single,"8,416",Traditional Residence Hall,Allen Hall,...,1,1,1,1,1,1,1,0,0,0
2,AH,NaN,Floor 1,103,1,1 bedroom,Traditional Single,"8,416",Traditional Residence Hall,Allen Hall,...,1,1,1,1,1,1,1,0,0,0
3,AH,NaN,Floor 1,104,1,1 bedroom,Traditional Single,"8,416",Traditional Residence Hall,Allen Hall,...,1,1,1,1,1,1,1,0,0,0
4,AH,NaN,Floor 1,105,1,1 bedroom,Traditional Single,"8,416",Traditional Residence Hall,Allen Hall,...,1,1,1,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552,JA,884- 899,Floor 2,888,2,2 bedrooms,Traditional Single,"9,960",Apartment,Johnson Apartments,...,1,1,1,1,1,1,1,1,0,1
1553,JA,884- 899,Floor 2,896,2,2 bedrooms,Traditional Single,"9,960",Apartment,Johnson Apartments,...,1,1,1,1,1,1,1,1,0,1
1554,JA,884- 899,Floor 2,897,2,2 bedrooms,Traditional Single,"9,960",Apartment,Johnson Apartments,...,1,1,1,1,1,1,1,1,0,1
1555,JA,884- 899,Floor 2,898,2,2 bedrooms,Traditional Single,"9,960",Apartment,Johnson Apartments,...,1,1,1,1,1,1,1,1,0,1


In [3]:
#Looking at all the attributes
print(df.columns)

Index(['hall_id', 'building_id', 'floor_no', 'room_no', 'occupancy',
       'room_type', 'floor_plan', 'price', 'type', 'name',
       'number of students', 'number of floors', 'average room size',
       'contract type', 'is_aircooled', 'has_vending_machine',
       'has_wifi_in_building', 'has_laundry', 'has_main_lounge',
       'has_floor_lounge', 'has_tv_in_building', 'has_recreational_activities',
       'has_communal_bathrooms', 'has_closet_space', 'has_desk', 'has_chair',
       'has_microfridge', 'has_curtains', 'has_tv_in_room', 'has_wifi_in_room',
       'has_carpeted_flooring', 'has_wardrobe', 'has_twin_bed_mattress',
       'has_hardwood_flooring', 'has_accessibility_ramps',
       'has_private_bathroom'],
      dtype='object')


In [4]:
#Drop unwanted featuers - features that are irrelevant or are constant across all the available rooms
features_to_drop = [
    'has_wifi_in_building','has_closet_space', 'has_communal_bathrooms','has_desk', 
    'has_chair','has_curtains', 'has_tv_in_room', 'has_wifi_in_room','has_microfridge',
    'has_carpeted_flooring', 'has_twin_bed_mattress','has_hardwood_flooring',
    'number of students' #hall specific attrribute not room specific
    ]

In [5]:
df.drop(features_to_drop, axis=1, inplace=True)

In [6]:
#Renaming columns
df.columns = ['hall_id', 'building_id', 'floor_number', 'room_number', 'max_room_occupancy',
       'room_type', 'floor_plan', 'price', 'residence_type', 'hall_name',
       'number_of_floors', 'average_room_size',
       'room_contract_type', 'is_aircooled', 'has_vending_machine', 'has_laundry',
       'has_main_lounge', 'has_floor_lounge', 'has_tv_in_building',
       'has_recreational_activities', 'has_wardrobe',
       'has_accessibility_ramps', 'has_private_bathroom']

In [7]:
#Check for null values
df.isnull().sum().to_frame()

,0
hall_id,0
building_id,618
floor_number,0
room_number,0
max_room_occupancy,0
room_type,0
floor_plan,0
price,0
residence_type,0
hall_name,0


In [8]:
#Sanity Check
df.loc[df['building_id'].isnull(), 'hall_id'].value_counts()

AH     117
BH     117
MH     117
MZH    117
CS      30
JS      30
MRS     30
TS      30
WS      30
Name: hall_id, dtype: int64

In [9]:
#floor_number : removing word 'floor' from the values
df['floor_number'] = df['floor_number'].str.replace("Floor ","")

In [10]:
# room_type : generalizing the word Bedroom(s)
df.loc[df.room_type == '1 bedroom', 'room_type'] = '1 bedrooms'
df['room_type'] = df['room_type'].str.replace('bedrooms','Bedroom(s)')

In [11]:
#number of floors : changing data type from floor to int
df[['number_of_floors']] = df[['number_of_floors']].astype(np.int32)

In [12]:
#average_room_size : Generalizing units of measure across data points
df.loc[df.average_room_size == "18 x 12", 'average_room_size'] = "216 sqft" 
df.loc[df.average_room_size == "11 x 18", 'average_room_size'] = "198 sqft"
df.loc[df.average_room_size == "273 sqft", 'average_room_size'] = "273 sqft"
df.loc[df.average_room_size == "12 x 16", 'average_room_size'] = "192 sqft"
df.loc[df.average_room_size == "9 x 15", 'average_room_size'] = "135 sqft"
df.loc[df.average_room_size == "10 x 11", 'average_room_size'] = "110 sqft"

In [13]:
#room_contract_type : removing Family Annual Contract and Graduate Academic Year – 9 Month Contracts (Two semesters and breaks) for sake of simplicity
df['room_contract_type'] = df['room_contract_type'].str.replace(', Family Annual Contract','')
df.loc[df.room_contract_type == "Graduate Full Calendar Year – 12 Month Contracts (Two semesters and all breaks), Graduate Academic Year – 9 Month Contracts (Two semesters and breaks)", 'room_contract_type'] = 'Graduate Full Calendar Year – 12 Month Contracts (Two semesters and all breaks)'

In [14]:
# Adding variable distance from student center
distances = {
    'BEST Suites' : '0.2 miles',
    'Nichols Apartments' : '0.7 miles',
    'Silvers Apartments' : '0.4 miles',
    'Richardson Apartments' : '0.7 miles',
    'Allen Hall' : '0.3 miles',
    'Barr Hall' : '0.3 miles',
    'Mattia Hall' : '0.3 miles',
    'Metzger Hall' : '0.3 miles',
    'Buell Apartments' : '0.2 miles',
    'McCormick Suites' : '0.2 miles',
    'Crosby Suites' : '0.3 miles',
    'Judson Suites' : '0.2 miles',
    'Morrow Suites' : '0.3 miles',
    'Thomas Suite' : '0.3 miles',
    'Winkler Suites' : '0.2 miles',
    'Johnson Apartments' : '0.5 miles',
}

col = 'distance_from_student_center'
for hall ,dist in distances.items():
    df.loc[df.hall_name == hall, col] = dist

In [15]:
df.price.value_counts()

8,416     468
9,366     452
8,968     244
8,652     197
9,960     100
11,618     96
Name: price, dtype: int64

In [16]:
# Changing Price data type from string to int
df['price'] = df['price'].str.replace(',','')
df['price'] = df['price'].astype('int')

In [17]:
#original distribution
df.groupby('hall_name').aggregate({'price':['mean'], 'max_room_occupancy':['sum']})

,price,max_room_occupancy
,mean,sum
hall_name,,
Allen Hall,8416.0,167
BEST Suites,8968.0,976
Barr Hall,8416.0,167
Buell Apartments,11618.0,384
Crosby Suites,8652.0,180
Johnson Apartments,9960.0,200
Judson Suites,8652.0,180
Mattia Hall,8416.0,223


In [18]:
#Unmodified Dataset : 
# Price Range for undergraduates : 8416 - 9366
# Price Range for graduates : 9960 - 11618
# Rule for changing price - Generate random number for undergraduate students by selecting n randon numbers from N(8000, 1500)
random.seed(100)
prices = np.random.normal(8000, 1500, 14).astype('int')
ug_hall = list(set(df['hall_name']) - set(['Buell Apartments','Johnson Apartments']))
hall_prices = dict(zip(ug_hall, prices))

In [19]:
for hall ,price in hall_prices.items():
    df.loc[df.hall_name == hall, 'price'] = price

In [20]:
# Modified Dataset price distribution
df.groupby('hall_name').aggregate({'price':['mean'], 'max_room_occupancy':['sum']})

,price,max_room_occupancy
,mean,sum
hall_name,,
Allen Hall,10177.0,167
BEST Suites,5880.0,976
Barr Hall,11177.0,167
Buell Apartments,11618.0,384
Crosby Suites,6128.0,180
Johnson Apartments,9960.0,200
Judson Suites,7480.0,180
Mattia Hall,6853.0,223


In [21]:
#Changing data so that each datapoint represents a singe bed and not an apartment in whole
rep_data = pd.DataFrame(df.values.repeat(df.max_room_occupancy, axis=0), columns=df.columns)

#function to get room_id
prev_hall_id = ''
prev_room_no = ''
bed_iter = 65 #A
for index, row in rep_data.iterrows():
    if prev_hall_id == row['hall_id'] and prev_room_no == row['room_number']:
        bed_iter+=1
    else:
        bed_iter = 65
        prev_hall_id = row['hall_id']
        # prev_building_id = row['building_id']
        prev_room_no = row['room_number']
    rep_data.loc[index,'room_id'] = row['room_number']+ "-" + chr(bed_iter)

In [22]:
# Reordering columns
rep_data = rep_data[[
    'hall_id','hall_name','residence_type','building_id','number_of_floors',
    'room_number','room_id','floor_number', 'floor_plan','room_type','max_room_occupancy',
    'distance_from_student_center','average_room_size','room_contract_type', 
    'is_aircooled', 'has_vending_machine','has_laundry', 'has_main_lounge', 
    'has_floor_lounge','has_tv_in_building', 'has_recreational_activities', 
    'has_wardrobe','has_accessibility_ramps', 'has_private_bathroom',
    'price']]

In [24]:
rep_data.to_csv('final_data.csv', index=False)